<a href="https://colab.research.google.com/github/yui828/hungman/blob/master/practice_NLP_%E5%BE%A9%E7%BF%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pythonで実践する自然言語処理

In [ ]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-locale-perl libfcgi-perl
  libhtml-parser-perl libhtml-tagset-perl libhttp-date-perl
  libhttp-message-perl libio-html-perl libio-string

In [ ]:
import MeCab
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import os
import numpy as np
import pandas as pd
import unicodedata

## データの読み込み
---
今回の利用するデータセットは、livedoor ニュースコーパスの一部です。

記事ファイル内容の提供元：  
ITライフハック(label:0) http://news.livedoor.com/category/vender/223/  
MOVIE ENTER(label:1) http://news.livedoor.com/category/vender/movie_enter/  
Peachy(label:2) http://news.livedoor.com/category/vender/ldgirls/  
Sports Watch(label:3) http://news.livedoor.com/category/vender/208/

In [ ]:
# データのダウンロード
!wget https://basic-unstructured-data.s3-ap-northeast-1.amazonaws.com/unstructured-data-day2/data.zip
!unzip data.zip

--2021-08-08 11:13:46--  https://basic-unstructured-data.s3-ap-northeast-1.amazonaws.com/unstructured-data-day2/data.zip
Resolving basic-unstructured-data.s3-ap-northeast-1.amazonaws.com (basic-unstructured-data.s3-ap-northeast-1.amazonaws.com)... 52.219.4.91
Connecting to basic-unstructured-data.s3-ap-northeast-1.amazonaws.com (basic-unstructured-data.s3-ap-northeast-1.amazonaws.com)|52.219.4.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4302872 (4.1M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   4.10M  4.75MB/s    in 0.9s    

2021-08-08 11:13:47 (4.75 MB/s) - ‘data.zip’ saved [4302872/4302872]

Archive:  data.zip
   creating: data/
  inflating: data/test.csv           
  inflating: data/train.csv          


In [ ]:
train = pd.read_csv("data/train.csv")
train.head()

,text,label
0,世界の映画祭で話題沸騰！インドネシアの壮絶バイオレンス・アクション『ザ・レイド』予告編が解禁...,1
1,冬の女子会は特製「薬膳火鍋」に決定！美味しく食べてポッカポカ本格的な寒さを感じるこの季節。「...,2
2,2012年・夏の浴衣スタイルのカギは帯！夏といえば浴衣。「そろそろ新しい浴衣を買おうかな」と...,2
3,インタビュー：武田修宏さん「結婚するならセリエAクラスの女性がいい」少年時代から天才サッカー...,2
4,「ポケモン」や「ONE PIECE」と同じ賞を獲得したキモかわいいキャラが全国に「クサマダラ...,1


In [ ]:
test = pd.read_csv("data/test.csv")
test.head()

,text,label
0,【Sports Watch】元代表二人によるカメルーン戦の予想は？TBSの報道番組「NEWS...,3
1,若槻千夏に激似!? 女子バレーのニューヒロイン新鍋理沙バレーボールのW杯が開幕し、ロンドン五...,3
2,ソーシャルの今が分かる！ヤフーの「話題なう」ってなに？ヤフーは、Twitterなどのソーシャ...,0
3,もう一つのユーロ2012美女ばかりの観客席には選手の彼女も!?連日連夜熱戦が展開されているサ...,3
4,元日本代表監督の岡田武史氏、杭州緑城の監督に決定日本代表の前監督である岡田武史氏が15日、中...,3


# 正規化
---

```In [1]: import unicodedata
In [2]: unicodedata.normalize("NFKC", "１②Ⅲ㈱㍗ﾊﾝｶｸ")
Out[2]: '12III(株)ワットハンカク'```

と同様の処理をPandas Seriesの値に対して実施する場合は、「Series.str.normalize」を利用します。  
「unicodedata.normalize」と同じ処理を実施してくれます。

In [ ]:
import unicodedata

In [ ]:
# ヒント
text = "私の誕生日は①月①①日のポッキーの日！!!?!"
unicodedata.normalize("NFKC", text)

'私の誕生日は1月11日のポッキーの日!!!?!'

### 問題1：train, testの列名textに対して、正規化してください。
ヒント：
df[列名].str.normalize("NFKC")

In [ ]:
# train["text"] = train["text"].str.normalize("NFKC")
# test["text"] = test["text"].str.normalize("NFKC")

def textnormalize(text):
  return unicodedata.normalize("NFKC", text)

train["text"] = train["text"].apply(textnormalize)
test["text"] = test["text"].apply(textnormalize)

In [ ]:
train["text"] = train["text"].str.normalize("NFKC")
test["text"] = test["text"].str.normalize("NFKC")

In [ ]:
train.head()

,text,label
0,世界の映画祭で話題沸騰!インドネシアの壮絶バイオレンス・アクション『ザ・レイド』予告編が解禁...,1
1,冬の女子会は特製「薬膳火鍋」に決定!美味しく食べてポッカポカ本格的な寒さを感じるこの季節。「...,2
2,2012年・夏の浴衣スタイルのカギは帯!夏といえば浴衣。「そろそろ新しい浴衣を買おうかな」と...,2
3,インタビュー:武田修宏さん「結婚するならセリエAクラスの女性がいい」少年時代から天才サッカー...,2
4,「ポケモン」や「ONE PIECE」と同じ賞を獲得したキモかわいいキャラが全国に「クサマダラ...,1


In [ ]:
len(train)

2785

In [ ]:
train["text"][992][:50]

'【終了しました】“女子力”のバイブル・映画『ガール』女性限定試写会に10組20名様をご招待映画『ガー'

## その他前処理
---
### 表記揺れの統一

### 問題2：train, testに"【】"と"『』"が混在しているので、"「」"で統一してください。

In [ ]:
# 以下のような文章をイメージしてください
print(train["text"][990][:50])

『ミッション:8ミニッツ』繰り返される“8分間”の悪夢に隠された真実とは全米でこの春公開され、観る者


In [ ]:
# ヒント
unify_dic = {
    '『': '「',
    '』': '」',
    '【': '「',
    '】': '」'
}
text = train["text"][992][:50]

# 対応表を作ることができる
dic_for_unify = str.maketrans(unify_dic)
text.translate(dic_for_unify)

'「終了しました」“女子力”のバイブル・映画「ガール」女性限定試写会に10組20名様をご招待映画「ガー'

In [ ]:
# 関数化しておきましょう
def unify_str(sentence):
    dic_for_unify = str.maketrans(unify_dic)
    sentence = sentence.translate(dic_for_unify)
    return sentence

unify_str(train["text"][992][:50])

'「終了しました」“女子力”のバイブル・映画「ガール」女性限定試写会に10組20名様をご招待映画「ガー'

In [ ]:
# Enter your code here
train["text"] = train["text"].apply(unify_str)
test["text"] = test["text"].apply(unify_str)

# 形態素解析
---

MeCab を利用した形態素解析

### 問題3：train, testの各文書を形態素解析してください。

In [ ]:
# ヒント
mecab = MeCab.Tagger()

# バグ回避用(おまじない)
mecab.parse("")

result = []

# node が None になるまで、node.next を辿り続ける
node = mecab.parseToNode("本日も晴天なり")
while node:
    result.append(node.surface)
    result.append(":")
    result.append(node.feature.split(",")[0])
    result.append(",")
    node = node.next

" ".join(result)

' : BOS/EOS , 本日 : 名詞 , も : 助詞 , 晴天 : 名詞 , なり : 助動詞 ,  : BOS/EOS ,'

In [ ]:
# 関数化しておきましょう
def get_morpheme(text):
    result = []
    node = mecab.parseToNode(text)
    while node:
        result.append(node.surface)
        node = node.next
    return " ".join(result)


get_morpheme("本日も晴天なり。")

' 本日 も 晴天 なり 。 '

In [ ]:
# Enter your code here
# トークン化する

mecab = MeCab.Tagger()
# バグ回避用
mecab.parse("")

train["text_tokenized"] = train["text"].apply(get_morpheme)
test["text_tokenized"] = test["text"].apply(get_morpheme)

**Janomeを利用した形態素解析**

In [21]:
!pip install janome

     |████████████████████████████████| 19.7 MB 75.9 MB/s 


In [23]:
from janome.tokenizer import Tokenizer

In [24]:
t = Tokenizer(wakati=False)

def get_morpheme_j(text):
  result_j = []
  for token in t.tokenize(text):
    partOfSpeech = token.part_of_speech.split(",")[0]
    if partOfSpeech not in ["助詞", "助動詞", "記号"]:
      result_j.append(token.surface)
  " ".join(result_j)

train["text_tokenized_j"] = train["text"].apply(get_morpheme)
test["text_tokenized_j"] = test["text"].apply(get_morpheme)

In [25]:
train.head()

,text,label,text_tokenized,text_tokenized_j
0,世界の映画祭で話題沸騰!インドネシアの壮絶バイオレンス・アクション「ザ・レイド」予告編が解禁...,1,世界 の 映画 祭 で 話題 沸騰 ! インドネシア の 壮絶 バイオレンス・アクション ...,世界 の 映画 祭 で 話題 沸騰 ! インドネシア の 壮絶 バイオレンス・アクション ...
1,冬の女子会は特製「薬膳火鍋」に決定!美味しく食べてポッカポカ本格的な寒さを感じるこの季節。「...,2,冬 の 女子 会 は 特製 「 薬 膳 火 鍋 」 に 決定 ! 美味しく 食べ て ポッ...,冬 の 女子 会 は 特製 「 薬 膳 火 鍋 」 に 決定 ! 美味しく 食べ て ポッ...
2,2012年・夏の浴衣スタイルのカギは帯!夏といえば浴衣。「そろそろ新しい浴衣を買おうかな」と...,2,2012 年 ・ 夏 の 浴衣 スタイル の カギ は 帯 ! 夏 と いえ ば 浴衣 。...,2012 年 ・ 夏 の 浴衣 スタイル の カギ は 帯 ! 夏 と いえ ば 浴衣 。...
3,インタビュー:武田修宏さん「結婚するならセリエAクラスの女性がいい」少年時代から天才サッカー...,2,インタビュー : 武田 修 宏 さん 「 結婚 する なら セ リエ A クラス の 女性...,インタビュー : 武田 修 宏 さん 「 結婚 する なら セ リエ A クラス の 女性...
4,「ポケモン」や「ONE PIECE」と同じ賞を獲得したキモかわいいキャラが全国に「クサマダラ...,1,「 ポケモン 」 や 「 ONE PIECE 」 と 同じ 賞 を 獲得 し た キモ か...,「 ポケモン 」 や 「 ONE PIECE 」 と 同じ 賞 を 獲得 し た キモ か...


### 問題4：形態素解析した後に、['助詞', '助動詞', '記号'] のみ除外してください。

In [26]:
# ヒント
mecab = MeCab.Tagger()

# バグ回避用
mecab.parse("")

result = []
node = mecab.parseToNode("本日も晴天なり。")
while node:
    if not node.feature.startswith("BOS/EOS") \
   and not node.feature.startswith("助詞") \
   and not node.feature.startswith("助動詞") \
   and not node.feature.startswith("記号"):
        result.append(node.surface)
    node = node.next

print(result)
" ".join(result)

['本日', '晴天']


'本日 晴天'

In [27]:
# 関数化しておきましょう
def get_surfaces_list(text):
    result = []
    node = mecab.parseToNode(text)
    while node:
        if node.feature.startswith("名詞"):
            result.append(node.surface)
        node = node.next
    return result

train['text_tokenized_list'] = train['text'].apply(get_surfaces_list)
all_token = pd.Series(train['text_tokenized_list'].sum())
all_token.value_counts()[0:20]

(     9697
の     7762
こと    7667
)     6917
日     6208
!     5508
:     5408
人     4636
1     4525
月     4453
-     4432
2     4412
3     4324
映画    4272
よう    3631
年     3532
.     3190
ん     2707
?     2700
中     2600
dtype: int64

In [ ]:
# Enter your code here
mecab = MeCab.Tagger()
# バグ回避用
mecab.parse("")

train["text_tokenized"] = train["text"].apply(get_surfaces_list)
test["text_tokenized"] = test["text"].apply(get_surfaces_list)

In [ ]:
train["text_tokenized"].head()

0    [世界, 映画, 祭, 話題, 沸騰, !, インドネシア, 壮絶, バイオレンス・アクショ...
1    [冬, 女子, 会, 特製, 薬, 膳, 火, 鍋, 決定, !, ポッカポカ, 本格, 的...
2    [2012, 年, 夏, 浴衣, スタイル, カギ, 帯, !, 夏, 浴衣, 浴衣, 人,...
3    [インタビュー, :, 武田, 修, 宏, さん, 結婚, セ, リエ, A, クラス, 女...
4    [ポケモン, ONE, PIECE, 賞, 獲得, キモ, キャラ, 全国, クサマダラオオ...
Name: text_tokenized, dtype: object

## その他前処理
---
### ありふれてて特徴が薄い単語を削除

"その"や"ため"のような文字は、特徴が薄い単語だと考えることができるので除去する。

### 問題5："その"や"ため"を、train, testの各文書から除外してください。

In [ ]:
# ヒント
text = train["text"][100][:50]
print(text)

stop_words = ["その", "ため"] # 適当な文字を設定
for s in stop_words:
    text = text.replace(s, "")

print(text)

マンガのプロが選ぶ「女子の心にグッとくる10の名セリフ」 / 女子のためのマンガ特集いつでもガールズ
マンガのプロが選ぶ「女子の心にグッとくる10の名セリフ」 / 女子ののマンガ特集いつでもガールズ


In [ ]:
train["text_tokenized"][150][:70]

In [ ]:
# 関数化しておきましょう
def remove_stop_words(sentence):
    stop_words = ["その", "ため"] # 適当な文字を設定
    for s in stop_words:
        sentence = sentence.replace(s, '')
    return sentence

In [ ]:
# Enter your code here
train["text_tokenized"] = train["text_tokenized"].apply(remove_stop_words)
test["text_tokenized"] = test["text_tokenized"].apply(remove_stop_words)

AttributeError: ignored

# 数値表現化
---

### 問題6：CountVectorizerでtrain, testを単語文書行列に変換してください。

※注意※  
単語文章行列を学習データ, テストデータでそれぞれ生成します。  
学習データに対してはfit_transform、そしてテストデータに対してはtransformを使用します。

In [ ]:
# ヒント
# token_pattern は、デフォルトだと1文字の単語を除外するので、除外しないように設定する
vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w+\b")

# スペース区切りの文書をリストで与える
X = vectorizer.fit_transform([
    "This is a pen",
    "I am not a pen",
    "Hello"
])

In [ ]:
X

In [ ]:
X.toarray()

In [ ]:
# これは、演習用に単語文書行列を DataFrame に変換して見やすくしてみるためのコードで、覚える必要はありません
pd.DataFrame(X.toarray(), columns=[ x[0] for x in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]) ])

In [ ]:
# Enter your code here
vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
train_mat = vectorizer.fit_transform(train["text_tokenized"])
test_mat = vectorizer.transform(test["text_tokenized"])

vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w+\b")

X_train = vectorizer.fit_transform(train["text_tokenized"])
X_test = vectorizer.transform(test["text_tokenized"])

### 問題7：TfidfVectorizerを使って、train, testを単語文書行列(TF-IDF適用済み)に変換してください。

※注意※  
単語文章行列を学習データ, テストデータでそれぞれ生成します。  
学習データに対してはfit_transform、そしてテストデータに対してはtransformを使用します。

In [ ]:
# ヒント
# token_pattern は、デフォルトだと1文字の単語を除外するので、除外しないように設定する
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")

# スペース区切りの文書をリストで与える
X = vectorizer.fit_transform([
    "This is a pen",
    "I am not a pen",
    "Hello"
])

In [ ]:
X.toarray()

In [ ]:
# これは、演習用に単語文書行列を DataFrame に変換して見やすくしてみるためのコードで、覚える必要はありません
pd.DataFrame(X.toarray(), columns=[ x[0] for x in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]) ])

In [ ]:
# Enter your code here
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")

X_train_tridf = vectorizer.fit_transform(train["text_tokenized"])
X_test_tridf = vectorizer.transform(test["text_tokenized"])

## 分析・機械学習
---
文書のベクトル化ができたら、後はこれまでやってきた機械学習と同様です。

いくつかの分類器で、性能を比較してみましょう。

In [ ]:
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
# Accuracy, Precision/Recall/F-score/Support, Confusion Matrix を表示
def show_evaluation_metrics(y_true, y_pred):
    print("Accuracy:")
    print(accuracy_score(y_true, y_pred))
    print()
    
    print("Report:")
    print(classification_report(y_true, y_pred))
    
    print("Confusion matrix:")
    print(confusion_matrix(y_true, y_pred))

### 問題8：作成したランダムフォレストを用いて、TfidfVectorizerで数値化したtestの予測をしてください。

In [ ]:
# ヒント：ロジスティック回帰モデルを作成する場合
clf_lr = LogisticRegression(n_jobs=-1)
clf_lr.fit(X_train_tridf, train["label"])

y_pred = clf_lr.predict(X_test_tridf)
show_evaluation_metrics(y_pred, test["label"])

In [ ]:
# Enter your code here
clf_lr = RandomForestClassifier(n_jobs=-1)
clf_lr.fit(X_train_tridf, train["label"])

y_pred = clf_lr.predict(X_test_tridf)
show_evaluation_metrics(y_pred, test["label"])

## 解答例

### 問題1：train, testの列名textに対して、正規化してください。
ヒント：
df[列名].str.normalize("NFKC")

In [ ]:
train["text"] = train["text"].str.normalize("NFKC")
test["text"] = test["text"].str.normalize("NFKC")

### 問題2：train, testに"【】"と"『』"が混在しているので、"「」"で統一してください。

In [ ]:
def unify_str(sentence):
    dic_for_unify = str.maketrans(unify_dic)
    sentence = sentence.translate(dic_for_unify)
    return sentence

In [ ]:
# Enter your code here
train["text"] = train["text"].apply(unify_str)
test["text"] = test["text"].apply(unify_str)

### 問題3：train, testの各文書を形態素解析してください。

In [ ]:
def get_morpheme(text):
    result = []
    node = mecab.parseToNode(text)
    while node:
        result.append(node.surface)
        node = node.next
    return " ".join(result)

get_morpheme("本日も晴天なり。")

In [ ]:
mecab = MeCab.Tagger()
# バグ回避用
mecab.parse("")

train['text_tokenized'] = train['text'].apply(get_morpheme)
test['text_tokenized'] = test['text'].apply(get_morpheme)

In [ ]:
train.head()

### 問題4：形態素解析した後に、['助詞', '助動詞', '記号'] のみ除外してください。

In [ ]:
def get_surfaces(text):
    result = []
    node = mecab.parseToNode(text)
    while node:
        if not node.feature.startswith("BOS/EOS") and not node.feature.startswith("助詞") and not node.feature.startswith("助動詞"):
            result.append(node.surface)
        node = node.next
    return " ".join(result)

get_surfaces("本日も晴天なり。")

In [ ]:
mecab = MeCab.Tagger()
# バグ回避用
mecab.parse("")

train['text_tokenized'] = train['text'].apply(get_surfaces)
test['text_tokenized'] = test['text'].apply(get_surfaces)

In [ ]:
train.head()

### 問題5："その"や"ため"を、train, testの各文書から除外してください。

In [ ]:
def remove_stop_words(sentence):
    stop_words = ["その", "ため"] # 適当な文字を設定
    for s in stop_words:
        sentence = sentence.replace(s, '')
    return sentence

In [ ]:
train["text_tokenized"] = train["text_tokenized"].apply(remove_stop_words)
test["text_tokenized"] = test["text_tokenized"].apply(remove_stop_words)

### 問題6：CountVectorizerでtrain, testを単語文書行列に変換してください。

※注意※  
単語文章行列を学習データ, テストデータでそれぞれ生成します。  
学習データに対してはfit_transform、そしてテストデータに対してはtransformを使用します。

In [ ]:
vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
train_mat = vectorizer.fit_transform(train["text_tokenized"])
test_mat = vectorizer.transform(test["text_tokenized"])

### 問題7：TfidfVectorizerを使って、train, testを単語文書行列(TF-IDF適用済み)に変換してください。

※注意※  
単語文章行列を学習データ, テストデータでそれぞれ生成します。  
学習データに対してはfit_transform、そしてテストデータに対してはtransformを使用します。

In [ ]:
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
train_mat = vectorizer.fit_transform(train["text_tokenized"])
test_mat = vectorizer.transform(test["text_tokenized"])

In [ ]:
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
def show_evaluation_metrics(y_true, y_pred):
    print("Accuracy:")
    print(accuracy_score(y_true, y_pred))
    print()
    
    print("Report:")
    print(classification_report(y_true, y_pred))
    
    print("Confusion matrix:")
    print(confusion_matrix(y_true, y_pred))

### 問題8：作成したランダムフォレストを用いて、TfidfVectorizerで数値化したtestの予測をしてください。

In [ ]:
clf_rf = RandomForestClassifier(n_estimators=10, n_jobs=-1)
clf_rf.fit(train_mat, train["label"])

y_pred = clf_rf.predict(test_mat)
show_evaluation_metrics(y_pred, test["label"])